In [2]:
import os
import sys
import pandas as pd
from xbbg import blp
sys.path.append('../src')
import utils

etf_ticker = 'EFA'
asof = '2025-09-25'

hold_df = pd.read_csv(f'cached_holdings/{etf_ticker}_{asof}.csv')

In [3]:
filtered = hold_df[hold_df['adr'].notnull() & (hold_df['adr_exchange'] != 'OOTC') & (hold_df['adr_turnover'] > 5e6)].copy()

In [4]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG','AUSTRALIA']),'index_future_bbg'] = 'Z'
filtered.loc[filtered['country'].isin(['JAPAN']),'index_future_bbg'] = 'NH'
filtered.loc[filtered['index_future_bbg'].isna(),'index_future_bbg'] = 'VG'

In [5]:
filtered.loc[filtered['exchange'] == 'MTAA', 'exchange'] = 'XMIL'

In [6]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG']),'market_etf_hedge'] = 'EWU'
filtered.loc[filtered['country'].isin(['JAPAN']),'market_etf_hedge'] = 'EWJ'
filtered.loc[filtered['country'].isin(['AUSTRALIA']),'market_etf_hedge'] = 'EWA'
filtered.loc[filtered['market_etf_hedge'].isna(),'market_etf_hedge'] = 'FEZ'

In [10]:
filtered[filtered['country']=='AUSTRALIA'][['id','adr','turnover_usd','adr_turnover']]

,id,adr,turnover_usd,adr_turnover
274,BHP AU Equity,BHP US Equity,2.539230e+08,1.667997e+08
682,WDS AU Equity,WDS US Equity,8.251675e+07,1.136334e+07


currency
GBp    22
EUR    17
JPY     7
AUD     2
DKK     2
CHF     2
NOK     1
SEK     1
ILs     1
Name: count, dtype: int64

In [8]:
filtered[filtered['currency'].isin(['AUD','JPY'])]['adr_turnover'].sum()

543300274.1384615

In [9]:
filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])].to_csv('../data/raw/all_adr_filtered.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','JPY'])].to_csv('../data/raw/adr_top_three_currencies.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR'])].to_csv('../data/raw/adr_gbp_eur.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])].to_csv('../data/raw/all_europe.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK','AUD','JPY'])].to_csv('../data/raw/all_europe_asia.csv', index=False)

In [ ]:
filtered[['country','exchange']].value_counts()

country      exchange
BRITAIN      XLON        21
JAPAN        XTKS         7
NETHERLANDS  XAMS         4
GERMANY      XETR         3
AUSTRALIA    XASX         2
SPAIN        XMAD         2
SWITZERLAND  XSWX         2
FRANCE       XPAR         2
DENMARK      XCSE         2
HONG KONG    XLON         1
IRELAND      XDUB         1
BELGIUM      XBRU         1
ITALY        XMIL         1
LUXEMBOURG   XMIL         1
FINLAND      XHEL         1
NETHERLANDS  XBRU         1
NORWAY       XOSL         1
SWEDEN       XSTO         1
ISRAEL       XTAE         1
Name: count, dtype: int64

In [8]:
filtered[filtered['currency'].isin(['GBp','EUR'])]['adr_turnover'].sum()/filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])]['adr_turnover'].sum()

0.7655713455345836

In [ ]:
filtered[filtered['country']=='D']['adr_turnover'].sum()

178163000.0153846

In [27]:
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])]['exchange'].value_counts()

exchange
XLON    22
XAMS     4
XETR     3
XBRU     2
XMAD     2
XMIL     2
XCSE     2
XSWX     2
XPAR     2
XOSL     1
XSTO     1
XHEL     1
XDUB     1
Name: count, dtype: int64

In [29]:
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])]['exchange'].value_counts()

exchange
XLON    22
XAMS     4
XETR     3
XBRU     2
XMAD     2
XMIL     2
XCSE     2
XSWX     2
XPAR     2
XOSL     1
XSTO     1
XHEL     1
XDUB     1
Name: count, dtype: int64

In [39]:
import pandas_market_calendars as mcal

mcal.get_calendar('XCSE').close_time

datetime.time(17, 0, tzinfo=zoneinfo.ZoneInfo(key='Europe/Copenhagen'))

In [43]:
for t in filtered[filtered['currency'].isin(['SEK','NOK','CHF','DKK'])]['adr'].str.replace(' US Equity','').tolist():
    print(f'- {t}')

- EQNR
- ERIC
- GMAB
- NVS
- NVO
- UBS


In [ ]:
xosl 5
xsto 0
xswx 1
xcse 0